{: .chap:serialization }

# Serialización

El único modo posible de procesar y almacenar datos en un computador actual es el código binario.
Pero, salvo unas pocas excepciones, rara vez resulta suficientemente expresivo para representar información útil para las personas.
Por ese motivo se utilizan distintas formas de interpretar el código binario en función del tipo de dato que se desea:
entero, decimal, cadena de caracteres, fecha, etc.
Lo importante es recordar que, sea cual sea su representación en un lenguaje de programación de alto nivel determinado, en la memoria o registros de las computadoras, todo dato es a fin de cuentas una secuencia de bits.

La serialización es el proceso de codificar los datos que manejan los programas
(enteros, cadenas, imágenes, etc.) en secuencias de bytes susceptibles de ser almacenadas en un fichero o
enviadas a través de la red.
Por el contrario, la des-serialización es el proceso inverso. Más concretamente, estamos tratando lo «serialización binaria». También existen muchos formatos de serialización textual como XML, JavaScript Object Notation (JSON), YAML Ain’t Markup Language (YAML), etc., que no vamos a tratar en este capítulo.

Es importante aclarar que no se debe confundir «codificación» con «cifrado» (o «encriptación»). Codificar es simplemente aplicar una transformación que modifica el modo en que se representan los datos, pero no implica el uso de ninguna clave ni ocultación u ofuscación del mensaje. Por ejemplo, un mensaje codificado en Morse es perfectamente legible por cualquiera que conozca el código


## Representación, sólo eso

Una de las excepciones a las que alude la sección anterior es la _programación de sistemas_, es decir, aquellos programas que consumen directamente servicios del SO.
El binario resulta útil para manejar campos de bits, banderas binarias o máscaras,
muy comunes cuando se manipulan registros de control, operaciones de E/S, etc.
Por eso a veces es necesario manipular datos con estas representaciones.

Sin embargo, como las personas, la mayoría de los lenguajes de programación utilizan la base decimal para expresar todo tipo de cantidades. Pero también ofrecen mecanismos para realizar cambios de base.

Python permite expresar literales numéricos en varios formatos.A continuación, en todos los ejemplos se representa el número 42; y en todos los casos, si se asigna a una variable, se está creando un entero (tipo int) con el mismo valor.

In [1]:
decimal = 42
binary = 0b101010
octal = 0o52
hexadecimal = 0x2a

print(f"decimal: {decimal}\nbinary: {binary}\noctal: {octal}\nhexadecimal: {hexadecimal}")

decimal: 42
binary: 42
octal: 42
hexadecimal: 42


Asimismo ofrece funciones para convertir entre bases: `bin()`, `oct()`, y `hex()`,
aunque se debe tener en cuenta que estas funciones únicamente devuelven cadenas (`str`)
y se utilizan precisamente para ofrecer _representaciones_ diferentes del mismo dato.
Observa que la salida de cada ejecución incluye unas comillas,
lo que denota que lo impreso es una cadena y no un número.

In [2]:
bin(42)

'0b101010'

In [3]:
oct(42)

'0o52'

In [4]:
hex(42)

'0x2a'

Además, el constructor de la clase `int` acepta una cadena de caracteres que represente un número, y opcionalmente un segundo argumento para indicar la base:

In [5]:
print(int("42"))
print(int("101010", 2))
print(int("52", 8))
print(int("2a", 16))
print(int("1J", 23))

42
42
42
42
42


Todo programador debe tener claro que `'42'`, `'101010'`, `'052'` o `'0x2A'` no son más que diferentes maneras
de representar el mismo dato, y que internamente el computador lo va a almacenar exatamente igual: en binario. Igual de importante es entender que es totalmente diferente alamacenar (o enviar) el entero que alguna de sus representaciones. Esto es fuente de innumerables confusiones para los programadores novatos.

## Los enteros en Python

El tipo más simple de cualquier lenguaje de programación suele ser `byte`,
que se corresponde con una secuencia de 8 bits.
Suele ser un entero sin signo, es decir, puede representar números enteros en el rango [0, 255].
El lenguaje Python, por su naturaleza dinámica, solo tiene un tipo de datos para enteros (`int`).
En Python un entero puede ser arbitrariamente largo puesto que el intérprete se encarga
de gestionar la memoria necesaria:

In [6]:
googol = 10 ** 100
type(googol)

int

En ocasiones, como cuando serializamos datos en un fichero o una conexión de red, necesitamos manejar explícitamente el tamaño de los datos.
En las próximas secciones veremos cómo realizar esa tarea en Python.

## Caracteres

La condificación de caracteres más simple (y una de las más antiguas) consiste en asignar un número a cada carácter del alfabeto.
**ASCII** fue creado por ANSI en 1963 como una evolución de la condificación utilizada anteriormente en telegrafía.
Es un código de 7 bits (128 símbolos) que incluye los caracteres alfanuméricos de la lengua inglesa,
incluyendo mayúsculas y minúsculas, y la mayoría de signos de puntuación y tipográficos habituales.
Además incluye caracteres de control para indicar salto de línea, de página, tabulador, retorno de carro, etc.
Más tarde IBM creó el código EBCDIC, similar a ASCII pero con 8 bits (256 símbolos).

Prácticamente todos los lenguajes de programación incluyen funciones elementales para manejar la conversión entre
bytes (números de 8 bits) y su carácter equivalente. En Python estas funciones son `ord()` y `chr()`:

In [7]:
ord("a")

97

In [8]:
chr(97)

'a'

In [14]:
ord("0")

48

In [9]:
ord("\0")

0

In [10]:
chr(0)

'\x00'

In [11]:
ord("\n")

10

In [12]:
chr(10)

'\n'

In [13]:
ord(" ")

32

Se puede ver en los ejemplos anteriores que, por ejemplo, el valor equivalente al carácter "0" es 48, mientras que el caracter equivalente al valor numérico 0 es el carácter "\x00".

Es especialemten importante tener claro que los caracteres numéricos **no son equivalentes** a los valores que representan. También resulta digno de mención que la secuencia '`\n`' es un sólo caracter, ya que la barra es lo que se conoce como **«carácter de escape»**. Es decir, cambia el significado del siguiente carácter. En este caso, significa «nueva línea», como ya sabéis.

De la misma manera, cuando un caracter toma un valor que no corresponde con ningún carácter imprimible, como en el caso anterior del caracter equivalente al número 0,
Python lo representa con '`\x00`', siendo los dos caracteres que siguen al `\x` el valor hexadecimal del número.

Como se explicó en la sesión de introducción a Python, las cadenas son un tipo inmutable, lo que implica que no puede modificarse su contenido.
Para añadir o modificar algún element en una cadena, debemos crear una nueva modificada a partir de la primera.

Sin embargo, Python proporciona el tipo **`bytearray`** que permite almacenar una secuencia de bytes, modificar su contenido, y obtener fácilmente la lista de caracteres o secuencia de enteros equivalente.

In [16]:
buf = bytearray("abcd", "ascii")
buf[0] = 20
print(f"buf: {buf}")
print(f"Decoded buf: {buf.decode()}")

buf: bytearray(b'\x14bcd')
Decoded buf: bcd


## Tipos multibyte y ordenamiento

En muchísimas ocasiones necesitamos utilizar nímeros enteros o reales que internamente van a ocupar más de un byte de memoria.
Ésto hace que aparezca un nuevo problema muy interesante: ¿cómo deben ordenarse los múltiples bytes que representan el dato?

Dependiendo de cómo se ordenen los bytes de un mismo dato entre ellos, se distingue entre _little endian_ y _big endian_.
_Little endian_ significa que el byte **menos significativo** se coloca primero en memoria (en la dirección más baja),
mientras que _big endian_ es justo al revés: el byte de **mayor peso** se coloca en la dirección más baja de memoria.

Esta ordenación o _byte order_ es característico de cada arquitectura hardware,
y es imprescindible que los programas manipulen los datos de manera adecuada para poder realizar operaciones con ellos.

Para comprobar qué tipo de arquitectura estás usando, pueden utilizar el módulo `sys` de Python:

In [18]:
import sys

sys.byteorder

'little'

En la red ocurre algo parecido: cuando colocamos un dato multibyte en "el cable" también se tiene que respetar,
al recibirlo en el otro extremo, el ordenamiento.
En particular los protocolos de la pila TCP/IP imponen que se utilice siempre el ordenamiento _big endian_.

Esto significa que, con bastante probabilidad, el equipo que estás manejando ahora mismo tiene un ordenamiento opuesto
al que utiliza la red a la que está conectado, por lo que deben convertirse los datos antes de ser enviados.
Para evitar que los programas tengan que comprobar directamente el orden de la arquitectura,
las librerías de conectividad (`socket`) proporcionan funciones que hacen la conversión por nosotros.

Evidentemente, cuando el mismo programa se ejecute en una arquitectura _big endian_, dicha función no hará nada.

- `socket.ntohs()`: convierte un entero _short_ (16 bits) del ordenamiento de la red al de la computadora.
- `socket.htons()`: convierte un entero _short_ (16 bits) del ordenamiento de la computadora al de la red.
- `socket.ntohl()`: convierte un entero _long_ (32 bits) del ordenamiento de la red al de la computadora.
- `socket.htonl()`: convierte un entero _long_ (32 bits) del ordenamiento de la computadora al de la red.

In [23]:
import socket

socket.htons(32)

8192

In [32]:
hex(32)

'0x20'

In [33]:
hex(socket.htons(32))

'0x2000'

Al ver la representación hexadecimal de 32 en esta máquina (_little endian_) y de su versión para red (_big endian_)
podemos observar lo siguiente:

- Cada byte se representa con 2 caracteres hexadecimales, cada caracter hexadecimal puede represnetar 16 valores.
- En la primera versión (_little endian_), el valor menos significativo aparece más a la derecha
    (los dos ceros que deberían estar a la izquierda se omiten, igual que haríamos en base 10).
- En la segunda representación (_big endian_) el valor menos significativo aparece más a la izquierda, y los más significativos más a la derecha.

## Cadenas de caracteres y secuencias de bytes

En Python 3 las cadenas de caracteres (tipo `str`) utilizan Unicode como codificación.
Ésto hace que, por defecto, no podamos utilizar `str` para escribir en un fichero o para enviar a través de un socket.
Estas operaciones requieren del uso de la secuencia de bytes (tipo `bytes`).
Convertir una cadena a una secuencia de bytes requiere que se indique una codificación o _encoding_.

Un _encoding_ es una tabla de correspondencia entre un caracter imprimible y su representación en uno o varios bytes.
Habitualmente los caracteres que ya estaban establecidos en el código ASCII tienen la misma representación en todos los _encodings_,
pero los caracteres no presentes en ASCII, como las vocales con las diferentes tildes, el símbolo del € o nuestra querida _ñ_
suelen tener diferentes representaciones dependiendo del _encoding_:

In [34]:
bytes("ñ", "utf-8")

b'\xc3\xb1'

In [36]:
bytes("ñ", "latin-1")

b'\xf1'

In [41]:
animal = "ñandú"
try:
    bytes(animal, "ascii")
except UnicodeEncodeError as ex:
    print(f"No se ha podido codificar {animal} usando ASCII:")
    print(ex)

print(f"{animal} tiene una longitud de {len(animal)}")

animal_raw = bytes(animal, "utf-8")
print(f"{animal_raw} tiene una longitud de {len(animal_raw)}")

No se ha podido codificar ñandú usando ASCII:
'ascii' codec can't encode character '\xf1' in position 0: ordinal not in range(128)
ñandú tiene una longitud de 5
b'\xc3\xb1and\xc3\xba' tiene una longitud de 7


En el primer intento de convertir la palabra "ñandú" usando ASCII obtenemos un error,
ya que ni la "ñ" ni la "ú" son caracteres contemplados en esta codificación.

Otro modo de poder convertir entre `bytes` y su representación en `str` y viceversa es a través de los métodos
`decode()` y `encode()` respectivamente:

In [42]:
"ñandú".encode()

b'\xc3\xb1and\xc3\xba'

In [43]:
b'\xc3\xb1and\xc3\xba'.decode()

'ñandú'

## Empaquetado

Hemos visto anteriormente cómo poder convertir números enteros de 16 y 32 bits o cadenas a su representación en secuencia de bytes.
Aunque el tipo `bytes` permite realizar múltiples operaciones, como la concatenación, partición o búsquedas,
cuando queremos realizar varias conversiones a la vez puede volverse muy engorroso.

Para ello Python proporciona el módulo `struct` que nos permite realizar transformaciones entre datos y su representación y viceversa.

En primer lugar, veremos como convertir un conjunto de datos a una secuencia de bytes equivalente utilizando `struct.pack()`.

### Cadena de formato

En primer lugar, la función `struct.pack()` debe conocer el tipo de datos, tamaño y ordenamiento
que queremos aplicar a cada uno de los valores que queramos convertir.
Para ello, el primer argumento que debe recibir `struct.pack()` es una cadena de formato.
Dicha cadena debe indicar, en su primera posición, el _byte order_ de acuerdo a los siguientes valores:

- `@`: ordenamiento nativo de la máquina realizando alineamiento.
- `=`: ordenamiento nativo.
- `<`: _little endian_.
- `>`: _big endian_.
- `!`: ordenamiento de la red (igual que el anterior).

A continuación, por cada valor que queramos "empaquetar" añadiremos un caracter más a la cadena de formato.
Just debajo se puede ver la documentación de `struct` y la explicación de cada valor de la cadena de formato:

In [44]:
import struct

print(struct.__doc__)

Functions to convert between Python values and C structs.
Python bytes objects are used to hold the data representing the C struct
and also as format strings (explained below) to describe the layout of data
in the C struct.

The optional first format char indicates byte order, size and alignment:
  @: native order, size & alignment (default)
  =: native order, std. size & alignment
  <: little-endian, std. size & alignment
  >: big-endian, std. size & alignment
  !: same as >

The remaining chars indicate types of args and must match exactly;
these can be preceded by a decimal repeat count:
  x: pad byte (no data); c:char; b:signed byte; B:unsigned byte;
  ?: _Bool (requires C99; if not available, char is used instead)
  h:short; H:unsigned short; i:int; I:unsigned int;
  l:long; L:unsigned long; f:float; d:double; e:half-float.
Special cases (preceding decimal count indicates length):
  s:string (array of char); p: pascal string (with count byte).
Special cases (only available in nati

### Ejemplo de empaquetado: cabecera de una trama Ethernet

Vamos a realizar un ejemplo en el que queremos empaquetar la cabecera de una trama Ethernet.
La cabecera Ethernet es muy sencilla: únicamente consta de la dirección destino de la trama,
la dirección origen y un entero corto (_short_).

Por ello, nuestra cadena de formato deberá ser `"!6s6sh"`:

- `!` para indicar que se utilice el ordenamiento de la red.
- `6s` para indicar que queremos pasarle 6 caracteres (cada una de las direcciones MAC).
- `h` para indicar que le pasamos un número entero que debe ser formateado como un _short_ con signo.

En concreto, queremos crear la cabecera de una trama Ethernet que contenga una petición ARP,
por lo qué la MAC destino deberá ser "FF:FF:FF:FF:FF:FF" y el tipo 0x0806:

In [47]:
header = struct.pack(
    "!6s6sh",
    b"\xff" * 6,  # Se concatena consigo misma 6 veces, produciendo la MAC de destino
    b"\xC4\x85\x08\xED\xD3\x07",  # una dirección MAC de origen, en este caso C4:85:08:ED:D3:07
    0x0806,
)

header

b'\xff\xff\xff\xff\xff\xff\xc4\x85\x08\xed\xd3\x07\x08\x06'

In [48]:
list(header)

[255, 255, 255, 255, 255, 255, 196, 133, 8, 237, 211, 7, 8, 6]

## Desempaquetado

Es el proceso opuesto al realizado en la sección anterior y se hace a través de la función `struct.unpack`.
En este caso, la funcón acepta 2 argumentos: la cadena de formato y la secuencia de bytes
de la que queremos desempaquetar los datos.

In [51]:
dst_address, orig_address, frame_type = struct.unpack("!6s6sh", header)
print(f"Destination address: {dst_address}")
print(f"Origin address: {orig_address}")
print(f"Frame type: {frame_type}")
print(f"Frame type (hex): {hex(frame_type)}")

Destination address: b'\xff\xff\xff\xff\xff\xff'
Origin address: b'\xc4\x85\x08\xed\xd3\x07'
Frame type: 2054
Frame type (hex): 0x806
